In [6]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) \
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [7]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [8]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [9]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOEKN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [10]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

@hotelkeys @Scavino45 @realDonaldTrump This is not suppose to be a fundraiser. Just when I thought Trump was as low https://t.co/0HljbGRETg
Long: None, Lati: None
@churchclarity maybe clarity for racism too ha
Long: None, Lati: None
RT @lawrencewagner0: @IlhanMN Keep pushing your hateful narrative ....

You are GUARANTEEING a TRUMP 2020 victory .. wait and see

I've see
Long: None, Lati: None
RT @SethAbramson: So Trump gave classified intelligence to Kremlin officials in the Oval Office and no Republicans cared, and then Jared Ku
Long: None, Lati: None
RT @CalebJHull: This is the most ridiculous thing Ive ever seen.

Trump broke these people.  https://t.co/xKVy1Bv31f
Long: None, Lati: None
And this is why I call you out Trumpers..Your foolish outcry over Castro...When Trump does this...Especially with t https://t.co/AOCG1c1E3N
Long: None, Lati: None
RT @Mediaite: MSNBC National Security Contributor Decodes Hitler Reference in Trump Lowering Flags for Shooting Victims https://t.co/Av8UI

RT @TelePolitics: Trump's trade wars make a no-deal Brexit increasingly dangerous for the EU | Ambrose Evans-Pritchard

https://t.co/rzhiKY
Long: None, Lati: None
RT @samstein: Ah yes, the courage to look at the difficult and accept it as impossible https://t.co/yraXnFYUjH
Long: None, Lati: None
RT @Scavino45: President @realDonaldTrump with the incredible medical staff at Miami Valley Hospital in Dayton, Ohio today. Some extremely
Long: None, Lati: None
@Kevy059 @DanaPerino NOPE, NOT LEFT but Trump who considers trad. Amer. values (like respect/love/compassion, integ https://t.co/k7gOY6HcAl
Long: None, Lati: None
RT @HuffPost: Trump is again trying to equate the ongoing threat of white supremacist violence with the antifa boogeyman. https://t.co/U2mr
Long: None, Lati: None
RT @JoeBiden: I wish I could say this hate began with Trump and will end with him. It didnt &amp; it wont.

The battle for the soul of this n
Long: None, Lati: None
RT @RyanAFournier: CNN contributor @rezaaslan is c

RT @indy3753: @JuddLegum someone needs to explain to the CEO how it all works. When the $$ goes up to the chairman, and out then out to Tru
Long: None, Lati: None
RT @TheRightMelissa: @realDonaldTrump 100% Proof Trump is A white Supremacist: 

Trump:
 1st Golf Club Owner in Palm Beach to allow Blacks
Long: None, Lati: None
@SaraCarterDC @CNN @MSNBC @nytimes @TheDemocrats Apparently the left's vision does not reach CHICAGO, it is so myop https://t.co/4MTWTOHQIG
Long: None, Lati: None
RT @AOC: Heres what we have to say to all of Americas men and women falling in the grips of hatred and white supremacy:

Come back. Its
Long: None, Lati: None
RT @TomFitton: This reminds me how the Obama IRS, FBI, DoJ targeted Tea Party and other conservatives (AND their donors) with illegal inves
Long: None, Lati: None
RT @SayWhenLA: Movie Showing Elites Hunting Down Trump Supporters Slated to Hit Theaters in September

At this point, if you are for expand
Long: None, Lati: None
RT @DiamondandSilk: Corey B

RT @nationalpost: Trump says he 'almost' got a good headline from the New York Times before Democrats went 'crazy' https://t.co/pr1BcVS0Eg
Long: None, Lati: None
RT @CarpeDonktum: You used your free speech to write a book that was publicly sold on Amazon.

Do you mind if I publish your home address?
Long: None, Lati: None
RT @RyanAFournier: CNN contributor @rezaaslan is calling for the eradication of Trump supporters.

Please RETWEET this to get this man FI
Long: None, Lati: None
RT @loral1957: HERE,HERE good for you! Im fromTEXAS and he is a great leader. He knows more about TEXANS  then that self proclaimed dictat
Long: None, Lati: None
RT @keithboykin: The Students for Trump founder just pleaded guilty to running a $46,000 fraud scam pretending to be a lawyer.

The Blacks
Long: None, Lati: None
RT @KatiePavlich: Keep doing this Democrats! Joaquin Castro outed one of his own donors in bid to shame Trump supporters, donor will no lon
Long: None, Lati: None
RT @Thomas1774Paine: Promine

@RealShennaFox @Scavino45 @realDonaldTrump Trump sucks, get over it.
Long: None, Lati: None
RT @JuliansRum: Brennan just listed the purpose of these false flags in sequential order:

- End the 2nd Amendment
- End the 1st Amendment
Long: None, Lati: None
RT @WalshFreedom: Donald Trump is the reason Donald Trump is not welcome in El Paso and Dayton today.
Long: None, Lati: None
@therealkevinTE @JRobertSeguin @NAudrey_ST @artytrace @56blackcat @auntieshay0304 @Samiknowsbest1 @QueenCandyLeigh https://t.co/ssqZ2idKet
Long: None, Lati: None
Oh Dear God!  Is this true? https://t.co/iyKgIAsHYn
Long: None, Lati: None
RT @realTuckFrumper: #TuckFrump NOT ABOUT HIM: Trump official crushed for bragging president was like a rock star while meeting shootin
Long: None, Lati: None
RT @Cajun_Chick_25: How can a sitting congressman post the names of 44 private Texas Citizens, business owners, whod donated the max to Pr
Long: None, Lati: None
RT @RepMarkMeadows: Right. Peter Strzok, the investigator weve 

...

RT @eugenegu: Republicans who are faux outraged over Joaquin Castro tweeting a list of publicly available names of people who donated the f
Long: None, Lati: None
RT @DonaldJTrumpJr: Donald Trump's no racist, as past acts and presidential record prove https://t.co/N6HC3JpGsc
Long: None, Lati: None
RT @BremainInSpain: This is who we are fighting for

Our children

Our grandchildren

Our great grandchildren

I refuse to let them grow up
Long: None, Lati: None
Time to report Trump's ACTIONS! His words are mostly 
Long: None, Lati: None
@bomani_jones Woooo, South Florida is reacting quickly on this one https://t.co/sbR6yFlgUN
Long: None, Lati: None
RT @GaryRayBetz: "The American right wing is enabling a dual crisis: gun violence and white supremacy, and Trump has lit a match to this po
Long: None, Lati: None
RT @realDonaldTrump: Trump Urges Unity Vs. Racism, was the correct description in the first headline by the Failing New York Times, but i
Long: None, Lati: None
RT @ananavarro: Liza, a

RT @RealChrisLittle: @MarkDice @sheppmrtx What kind of crack heads does @MSNBC hire? At this point how are they considered a main stream ne
Long: None, Lati: None
RT @ABCPolitics: Former Texas Rep. Beto O'Rourke participates in an #ElPasoStrong event ahead of President Trump's visit to the city after
Long: None, Lati: None
RT @GovHowardDean: Sorry Maggie, but this seems true. How do you square publishing the Clinton donor names and then condemning someone who
Long: None, Lati: None
RT @renato_mariotti: This is why public disclosure of campaign donations matters. @Equinox customers can decide whether they want their mon
Long: None, Lati: None
RT @marklevinshow: Anti-Semitic Democrat Virginia delegate who protested President Trump the other week was the same sleaze ball who dumped
Long: None, Lati: None
RT @Ilhan: Counties that hosted a 2016 Trump rally saw a 226% increase in hate crimes.

Assaults increase when cities host Trump rallies.
Long: None, Lati: None
RT @PaulaChertok: Republic

@realDonaldTrump 
Can you Say Liberalism is a Violent Mental Illness now??
Long: None, Lati: None
At 1st this seems repulsively inappropriate, until you remember its trumps brand.
Long: None, Lati: None
RT @TruNews: President Trump is performing his duties as comforter-in-chief today in Dayton, Ohio, and El Paso, Texas, to extend the nation
Long: None, Lati: None
RT @jawofsteel2014: The fuck? https://t.co/pId6V2ZQnt
Long: None, Lati: None
RT @BreitbartNews: UPDATE: Rep. Joaquin Castro (D-TX) targeted one of his own supporters in a public list of San Antonio, Texas, natives wh
Long: None, Lati: None
RT @Yamiche: President Trump is literally tweeting insults at former VP Joe Biden in between visiting families of people killed in mass sho
Long: None, Lati: None
RT @Cernovich: I wont dox people as @JoaquinCastrotx did, but can confirm that his target list omitted Latino donors in San Antonio for Tr
Long: None, Lati: None
RT @BrentScher: Joe Biden today: "Trump Has More in Common With Georg

RT @MZHemingway: The 'Hate Trump' agenda by the Democrats has gone way too far https://t.co/hafAyIJq9C
Long: None, Lati: None
@erykaarminio Makes sense you support Trump, I mean, you can barely spell.
Long: None, Lati: None
RT @agraybee: I dunno man, maybe Trump fucking up in Ohio will have 2020 implications?
Long: None, Lati: None
@Strandjunker @PeanutBrittle13 When Trump is out I believe we are in for quite an awakening at the level of corrupt https://t.co/tL6voWHpmQ
Long: None, Lati: None
RT @TWLadyGrey: @gtconway3d Its as if supporting trump is an embarrassing thing to do? 
Long: None, Lati: None
RT @SethAbramson: So Trump gave classified intelligence to Kremlin officials in the Oval Office and no Republicans cared, and then Jared Ku
Long: None, Lati: None
RT @hodgetwins: This is to funny Melissa  #KAG https://t.co/FC4SC3jG48
Long: None, Lati: None
RT @TomFitton: The Left is trying to outlaw opposition to open borders and the related invasion of hundreds of thousands across our uns

KeyboardInterrupt: 